In [ ]:
%matplotlib inline
import pytim
import MDAnalysis as mda
import numpy as np
from pytim.datafiles import pytim_data, WATERSMALL_GRO
from pytim.utilities import lap
from pytim.observables import Correlator, Velocity
from matplotlib import pyplot as plt
#import deepdiff
from pytim import utilities

In [ ]:
TRAJ = pytim_data.fetch('WATERSMALL_LONG_TRR')

In [ ]:
u = mda.Universe(WATERSMALL_GRO,TRAJ)
g = u.select_atoms('name OW ')
inter = pytim.ITIM(u,group=g,molecular=False)

In [ ]:
# Note: the group of which we want to compute the autocorrelations 
# will be passed later as an imput to the correlate() methot.

# velocity autocorrelation, consider all 3 spatial directions
total_vv = Correlator(observable=Velocity())
# velocity autocorrelation in the xy plane, use group g as a reference group 
layer_vv = Correlator(observable=Velocity('xy'), reference=g)
# survival probability
layer_nn = Correlator(reference=g)


for t in u.trajectory[1:4000]:
    if t.frame % 100 == 0:
        print t.frame,
    # velocity correlation of the whole set of oxygen atoms
    total_vv.sample(g)
    # velocity correlation of the surface atoms
    layer_vv.sample(inter.atoms)
    # survival probability in the first group
    layer_nn.sample(inter.atoms)

In [ ]:
# after the sampling of the time series is over, we compute the correlations:

vacf_total = total_vv.correlation()

# in the reentrant variant, leaving the layer does not interrupt the contribution
# to the autocorrelation function
vacf_layer = layer_vv.correlation(reentrant=True) 
vacf_layer_nr = layer_vv.correlation(reentrant=False) 
survival = layer_nn.correlation()

In [ ]:
lim = 500
time = u.trajectory[1].time * np.arange(lim)

plt.plot(time,vacf_total[:lim],label='total')
plt.plot(time,vacf_layer[:lim],'.',label='in-layer, reentrant')
plt.plot(time,vacf_layer_nr[:lim],'-',label='in-layer, non reentrant')
plt.plot(time,survival[:lim],label='survival probability, reentrant')
plt.plot(time,time*0,c='black')

plt.xlabel('time / ps')
plt.legend()
plt.show()

In [1]:
#%matplotlib inline
import pytim
import MDAnalysis as mda
import numpy as np
from pytim.datafiles import pytim_data, WATERSMALL_GRO
from pytim.utilities import lap
from pytim.observables import Correlator, Velocity
from matplotlib import pyplot as plt
#import deepdiff
from pytim import utilities
TRAJ = pytim_data.fetch('WATERSMALL_LONG_TRR')
u = mda.Universe(WATERSMALL_GRO,TRAJ)
g = u.select_atoms('name OW ')
inter = pytim.ITIM(u,group=g,molecular=False)

checking presence of a cached copy ... found 


In [2]:
#bv =  Correlator(observable=Velocity('xy'))
vv = Correlator(observable=Velocity('x'), reference=g[0:2])
vvv = Correlator(observable=Velocity('xy'), reference=g[0:2])
nn = Correlator(reference=g[0:2])
g[0:2]

<AtomGroup with 2 atoms>

In [3]:
g[0:2].velocities*=0.0
g[0:2].velocities+=1.0

In [4]:
g[:2].velocities

array([[ 1.,  1.,  1.],
       [ 1.,  1.,  1.]], dtype=float32)

In [5]:
vv.sample(g[:2])
#bv.sample(g[:2])
vvv.sample(g[:2])
nn.sample(g[:2])

vv.sample(g[:1])
nn.sample(g[:1])
vvv.sample(g[:1])
#bv.sample(g[:2])

g[0:2].velocities/=2
vv.sample(g[:2])
nn.sample(g[:2])
vvv.sample(g[:2])
#bv.sample(g[:2])

#vv.sample(g[:2])
#nn.sample(g[:2])

#vv.sample(g[:1])
#nn.sample(g[:1])



In [28]:
np.set_printoptions()

a=nn.correlation(reduced=False,normalized=False,intermittent=True)
a

array([[ 1.   ,  0.667],
       [ 1.   , -0.   ],
       [ 1.   ,  1.   ]])

In [13]:
print vv.timeseries
print nn.maskseries

[[1.0, 1.0], [1.0, -0.0], [0.5, 0.5]]
[[True, True], [True, False], [True, True]]


In [20]:
a=[1.,2.,3./2.]

### TEST

In [ ]:
#u = mda.Universe(WATER_GRO)
g=u.atoms[0:2]
g.velocities*=0.0
g.velocities+=1.0
C = {}
C['vv-x'] = Correlator(observable=Velocity('x', reference=g)) 
C['vv-xy']= Correlator(observable=Velocity('xy', reference=g))
C['nn']   = Correlator(reference=g)

for name,c in C.iteritems():
    print 'sampling',name
    c.sample(g)
    c.sample(g[:1])
    c.sample(g)

print C['nn'].maskseries
print C['vv-x'].timeseries


In [ ]:
# print correlation_(nn,reduced=False,normalized=False,intermittent=True)
print correlation_(nn,reduced=True,normalized=False,intermittent=True)
print correlation_(nn,reduced=True,normalized=True,intermittent=True)

In [ ]:
correlation_intermittent(nn,reduced=False,normalized=False),"\n"
print nn.corr
print nn.norm
print np.average(nn.maskseries,axis=0)
print "---"
print nn.corr*nn.norm #/np.average(nn.maskseries,axis=0)
print 1+np.arange(1.*nn.norm.shape[0])[::-1]
print  nn.corr/(nn.norm/(1+np.arange(nn.norm.shape[0])[::-1]).reshape(3,1))
print (nn.norm/(1+np.arange(nn.norm.shape[0])[::-1]).reshape(3,1))

In [ ]:
print nn.maskseries
print nn.corr
print np.average(np.cumsum(nn.maskseries,axis=0),axis=1)[::-1]

In [ ]:
test = np.asarray(100*np.random.random(int(1e6)),dtype=np.int)
#print test
%timeit np.cumsum(test)[::-1]
%timeit 1.*np.cumsum(test)[::-1]
%timeit np.cumsum(test,dtype=np.float)[::-1]


In [ ]:
np.cumsum(test,dtype=np.float)

In [ ]:
np.double

In [ ]:
%timeit np.arange(int(1e5))
%timeit 1+np.arange(int(1e5))
%timeit 1.+np.arange(int(1e5))
%timeit 1.+np.arange(1.+1e5)[1:]


In [ ]:
%timeit test.reshape(1000000,1)

In [ ]:
print nn.maskseries

In [ ]:
np.fft.fft(nn.maskseries)

In [ ]:
size = len(nn.maskseries)
fa1 = np.fft.fft(nn.maskseries, axis=0, n= size * 2)
print fa1


In [ ]:
norm = np.arange(size)[::-1] + 1.
print np.fft.fft(fa1*np.conj(fa1),axis=0)[:size]
print norm

In [ ]:
print np.fft.fft(fa1*np.conj(fa1),axis=0).real[:size]/norm.reshape(norm.shape[0],1)/(2*size)

In [ ]:
print size

In [ ]:
print fa1.shape

In [ ]:
print norm

In [ ]:
print nn.nseries

In [ ]:
[False]*3

In [ ]:
print nn.maskseries

In [ ]:
print [[not i and i  for i in nn.maskseries[0]]]

In [ ]:
[[False for i in nn.maskseries[0]]]

In [ ]:
[[False] * len(nn.maskseries[0])]

In [ ]:
np.average(1,axis=0)

In [ ]:
print nn.maskseries
print np.average([nn.maskseries[0]],axis=1)

In [ ]:
a=np.array([1,2,3])

In [ ]:
a.shape

In [ ]:
a.reshape(1,3)

In [ ]:
a

In [ ]:
a/np.average(np.array([1]*3),axis=1)

In [ ]:
a=np.random.random(100000)

In [ ]:
%timeit np.fft.fft(a)
%timeit np.cumsum(a)

In [41]:
    def determine_dimension(self):
        self.nseries = max(len(self.timeseries),len(self.maskseries))

        if len(self.shape) == 1:
            shape = (self.nseries, self.shape[0], 1)
            dim = 1
        elif len(self.shape) == 2:
            shape = (self.nseries, self.shape[0] , self.shape[1])
            dim = self.shape[1]
        else:
            raise RuntimeError("Correlations of tensorial quantites not allowed in "+self.name)
        return dim
  
    def survival_probability(self,ms,normalized,reduced,intermittent):
        if intermittent == True:
            corr, norm = survival_intermittent(self,ms,normalized)
        else:
            corr, norm = survival_continuous(self,ms,normalized)
                
        if reduced == True:
            corr = np.average(corr,axis=1)
            if normalized == True:
                corr /= np.average(norm,axis=1)
        elif normalized == True:
            corr /= norm
            
        return corr
    

    def survival_intermittent(self,ms,normalized):
        norm  = None
        corr = utilities.correlate(ms)
        if normalized == True:
            norm = np.cumsum(ms,axis=0)[::-1]
            norm /= (1.+np.arange(norm.shape[0])[::-1]).reshape(norm.shape[0],1)
 
        return corr, norm

        
    def survival_continuous(self,ms,normalized):
        norm = None
        n_part = len(ms[0])
        corr = np.zeros((self.nseries,n_part))
        for part in range(n_part):
            edges = np.where(ms[::,part][:-1] != ms[::,part][1:])[0]
            deltat = edges[1::2]-edges[0::2]
            for dt in deltat: # for each of the disconnected segments
                              # no need to compute the correlation, we know what it is
                              # (already normalized...)
                corr[0:dt,part] += 1./len(deltat)
                
        return corr, np.array( [ [1] * n_part ] ) # to be consistent, will average to one
    
    
    def autocorrelation_continuous(self, ts, ms):

        dim = self.dim 
        norm = None
        n_part = len(ms[0])
        corr = np.zeros(ts.shape)
        for part in range(n_part):
            edges = np.where(ms[::,part][:-1] != ms[::,part][1:])[0]
            deltat = edges[1::2]-edges[0::2]
            for ind,dt in enumerate(deltat): # for each of the disconnected segments
                              # no need to compute the correlation, we know what it is
                              # (already normalized...)
                t1 = edges[2*ind]
                t2 = edges[2*ind+1]
                i1 = dim*part
                i2 = dim*(part+1)
                corr[0:dt,i1:i2] += utilities.correlate(ts[t1:t2,i1:i2])/len(deltat)
                
        return corr,corr[:,0] # to be consistent, will average to one
    
    def correlation_(self, reduced = False, normalized = True, exact = True, intermittent = True):
      
        self.dim = determine_dimension(self)

        # the standard correlation 
        if self.reference is None :
            corr = utilities.correlate(ts)
            norm = corr[:,0]
            if reduced == True :
                corr = np.average(corr,axis=1)
                norm = corr[0]
            if normalized == True:
                corr /= norm
            return corr

        # prepare the mask for the intermittent/continuous cases
        if intermittent == True:
            ms = np.asarray(self.maskseries,dtype=np.double)
        else: # we add Falses at the begining and at the end to ease the splitting in sub-trajectories
            falses = [[False] * len(nn.maskseries[0])]
            ms = np.asarray(falses+self.maskseries+falses)

        # compute the survival probabily
        if self.observable is None:  
            return survival_probability(self,ms,normalized,reduced,intermittent)
        # compute the autocorrelation function 
        else:
            ts = np.asarray(self.timeseries)
            return autocorrelation(self,ts, ms, normalized, reduced, intermittent)

    def  autocorrelation(self,ts, ms, normalized, reduced, intermittent):

        if intermittent == True:
            corr, norm = autocorrelation_intermittent(self,ts,ms)
        else:
            corr, norm = autocorrelation_continuous(self,ts,ms)
        if reduced == True:
            corr = np.average(corr,axis=1)
        if normalized == True:
            corr /= corr[0]  
            
        return corr

    
    def autocorrelation_intermittent(self,ts,ms):
        
        dim = self.dim

        
        maskcorr = utilities.correlate(ms)
        cond = np.where(maskcorr>1e-9)
        corr = ts.copy()
        for xyz in range(dim):
            corr[:,xyz::dim] = utilities.correlate(ts[:,xyz::dim]*ms)
            corr[:,xyz::dim][cond] /=  maskcorr[cond]
            #corr[:,xyz::dim][~cond] = 0.0

        norm = corr[:,0]
  
        return corr, norm
    

In [29]:
print np.around(-0.0,decimals=2)

-0.0


In [30]:
-0.0 == 0.0

True

In [38]:
vv.correlation(reduced=False,normalized=False,intermittent=  True)

array([[ 0.75 ,  0.625],
       [ 0.75 , -0.   ],
       [ 0.5  ,  0.5  ]], dtype=float32)

In [39]:
vv.correlation(reduced=True,normalized=False,intermittent=  True)

array([ 0.688,  0.375,  0.5  ], dtype=float32)

In [42]:
print      correlation_(vv, reduced = False, normalized = False, exact = True, intermittent = True)


[[ 0.75   0.625]
 [ 0.75  -0.   ]
 [ 0.5    0.5  ]]


In [47]:
print np.cumsum(nn.maskseries,axis=0)/np.arange(nn.maskseries.sa)[::-1]

[[3 2]
 [2 1]
 [1 1]]
